#### Gold Layer

In [3]:
today_date = '9999-99-99'  #'2025-01-06'
workspace = 'NA'            #"Fabric_DEV"


StatementMeta(, ce999be1-aee6-4e03-a99c-854c62784cff, 5, Finished, Available, Finished)

#### Reading Silver Table

In [3]:
from pyspark.sql.functions import col
df = spark.read.format('delta').load(f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_Silver.Lakehouse/Tables/silver_data").filter(col("Processing_Date") == str(today_date))

StatementMeta(, 00754576-9c74-4bba-b382-60deecc09c92, 5, Finished, Available, Finished)

In [4]:
df.printSchema()

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 6, Finished, Available, Finished)

root
 |-- Student_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = false)
 |-- Gender: string (nullable = false)
 |-- Grade_Level: string (nullable = true)
 |-- Course_ID: string (nullable = true)
 |-- Course_Name: string (nullable = true)
 |-- Enrollment_Date: date (nullable = true)
 |-- Completion_Date: date (nullable = true)
 |-- Status: string (nullable = false)
 |-- Final_Grade: string (nullable = false)
 |-- Attendance_Rate: double (nullable = false)
 |-- Time_Spent_on_Course_hrs: double (nullable = false)
 |-- Assignments_Completed: integer (nullable = false)
 |-- Quizzes_Completed: integer (nullable = false)
 |-- Forum_Posts: integer (nullable = false)
 |-- Messages_Sent: integer (nullable = false)
 |-- Quiz_Average_Score: double (nullable = false)
 |-- Assignment_Scores: string (nullable = true)
 |-- Assignment_Average_Score: double (nullable = false)
 |-- Project_Score: double (nullable = false)
 |-- Extra_Credit: double (nullable

### Creating Dimension Tables

#### Creating Dim_students Table

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from delta.tables import DeltaTable

# Define schemas for dimension tables
dim_student_schema = StructType([
    StructField("Student_ID", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Gender", StringType(), True),
    StructField("Demographic_Group", StringType(), True),
    StructField("Internet_Access", StringType(), True),
    StructField("Learning_Disabilities", StringType(), True),
    StructField("Preferred_Learning_Style", StringType(), True),
    StructField("Language_Proficiency", StringType(), True),
    StructField("Parent_Involvement", StringType(), True)
])

DeltaTable.createIfNotExists(spark).tableName("Dim_student")\
          .addColumns(dim_student_schema)\
          .execute()

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 7, Finished, Available, Finished)

#### Creating Dim_course Table

In [9]:
# Dim_Course Schema
dim_course_schema = StructType([
    StructField("Course_ID", StringType(), True),
    StructField("Course_Name", StringType(), True),
    StructField("Grade_Level", StringType(), True)
])

DeltaTable.createIfNotExists(spark).tableName('Dim_Course')\
          .addColumns(dim_course_schema)\
          .execute()

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 10, Finished, Available, Finished)

#### Creating Fact_Student_performance table

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

# Define schema for fact table
fact_student_performance_schema = StructType([
    StructField("Student_ID", StringType(), True),
    StructField("Course_ID", StringType(), True),
    StructField("Enrollment_Date", DateType(), True),
    StructField("Completion_Date", DateType(), True),
    StructField("Status", StringType(), False),
    StructField("Final_Grade", StringType(), False),
    StructField("Attendance_Rate", DoubleType(), False),
    StructField("Time_Spent_on_Course_hrs", DoubleType(), False),
    StructField("Assignments_Completed", IntegerType(), False),
    StructField("Quizzes_Completed", IntegerType(), False),
    StructField("Forum_Posts", IntegerType(), False),
    StructField("Messages_Sent", IntegerType(), False),
    StructField("Quiz_Average_Score", DoubleType(), False),
    StructField("Assignment_Scores", StringType(), True),
    StructField("Assignment_Average_Score", DoubleType(), False),
    StructField("Project_Score", DoubleType(), False),
    StructField("Extra_Credit", DoubleType(), False),
    StructField("Overall_Performance", DoubleType(), False),
    StructField("Feedback_Score", DoubleType(), False),
    StructField("Completion_Time_Days", IntegerType(), True),
    StructField("Performance_Score", DoubleType(), False),
    StructField("Course_Completion_Rate", StringType(), False),
    StructField("Processing_Date", DateType(), True)
])

DeltaTable.createIfNotExists(spark).tableName("fact_student_performance")\
           .addColumns(fact_student_performance_schema)\
           .execute()

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 8, Finished, Available, Finished)

#### Loading Data into Dim_student

In [10]:
df_selected_dim_student = (df.select( "Student_ID", 
    "Name", 
    "Age", 
    "Gender", 
    "Demographic_Group", 
    "Internet_Access", 
    "Learning_Disabilities", 
    "Preferred_Learning_Style", 
    "Language_Proficiency", 
    "Parent_Involvement"))

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 11, Finished, Available, Finished)

In [11]:
from delta.tables import *

dim_student_table_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_gold.Lakehouse/Tables/dim_student"
dim_deltastudent = DeltaTable.forPath(spark,dim_student_table_path)

## Perform the MERGE (UPSERT)

dim_deltastudent.alias('target').merge(
    df_selected_dim_student.alias('source'),
    "target.Student_ID = source.Student_ID"
).whenMatchedUpdate( set = {
    "Name": "source.Name",
    "Age": "source.Age",
    "Gender": "source.Gender",
    "Demographic_Group": "source.Demographic_Group",
    "Internet_Access": "source.Internet_Access",
    "Learning_Disabilities": "source.Learning_Disabilities",
    "Preferred_Learning_Style": "source.Preferred_Learning_Style",
    "Language_Proficiency": "source.Language_Proficiency",
    "Parent_Involvement": "source.Parent_Involvement"
}).whenNotMatchedInsert(values = {
    "Student_ID": "source.Student_ID",
    "Name": "source.Name",
    "Age": "source.Age",
    "Gender": "source.Gender",
    "Demographic_Group": "source.Demographic_Group",
    "Internet_Access": "source.Internet_Access",
    "Learning_Disabilities": "source.Learning_Disabilities",
    "Preferred_Learning_Style": "source.Preferred_Learning_Style",
    "Language_Proficiency": "source.Language_Proficiency",
    "Parent_Involvement": "source.Parent_Involvement"

}).execute()

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 12, Finished, Available, Finished)

In [12]:
# Get the history of the Delta table to extract metrics
history_df = dim_deltastudent.history(1)  # Get the latest operation

# Extract metrics from the history DataFrame
operation_metrics = history_df.select("operationMetrics").collect()[0][0]

# Extract specific metrics
rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted) 

print('Total rows of table: ',dim_deltastudent.toDF().count())
print("Merge Metrics:")
print(f"Rows inserted: {rows_inserted}")
print(f"Rows updated: {rows_updated}")
print(f"Rows deleted: {rows_deleted}")
print(f"Total rows affected: {rows_affected}")

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 13, Finished, Available, Finished)

Total rows of table:  15
Merge Metrics:
Rows inserted: 15
Rows updated: 0
Rows deleted: 0
Total rows affected: 15


In [13]:
%%sql

SELECT * FROM LH_gold.dim_student LIMIT 1000

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 14, Finished, Available, Finished)

<Spark SQL result set with 15 rows and 10 fields>

#### Loading data into Dim_course

In [2]:
# Selecting required column for Dim_course
df_selected_dim_course = (df.select( "Course_ID",
                                     "Course_Name",
                                     "Grade_Level"
                                    )
                         )

StatementMeta(, ce999be1-aee6-4e03-a99c-854c62784cff, 4, Finished, Available, Finished)

NameError: name 'df' is not defined

In [1]:
df_selected_dim_course = df_selected_dim_course.dropDuplicates(["Course_ID"])

# Alternative Solution 2: Pick the latest row based on some criteria (e.g., timestamp)
# Ensure you have a timestamp or similar field to order by (e.g., "Processing_Date")
# from pyspark.sql.functions import row_number, desc
# from pyspark.sql.window import Window

# window_spec = Window.partitionBy("Course_ID").orderBy(desc("timestamp"))  # Replace "timestamp" with actual field
# df_selected_dim_course = df_selected_dim_course.withColumn("row_number", row_number().over(window_spec))\
#                                                 .filter(col("row_number") == 1)\
#                                                 .drop("row_number")

# Alternative Solution 3: Aggregating the source data to eliminate duplicates
# from pyspark.sql import functions as F
# df_selected_dim_course = df_selected_dim_course.groupBy("Course_ID").agg(
#     F.first("Course_Name").alias("Course_Name"),
#     F.first("Grade_Level").alias("Grade_Level")
# )

# Perform the MERGE (UPSERT) operation
from delta.tables import *

dim_course_table_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_gold.Lakehouse/Tables/dim_course"
dim_deltacourse = DeltaTable.forPath(spark, dim_course_table_path)

dim_deltacourse.alias("target").merge(
    df_selected_dim_course.alias("source"),
    "target.Course_ID = source.Course_ID"
).whenMatchedUpdate(set={
    "Course_ID": "source.Course_ID",
    "Course_Name": "source.Course_Name",
    "Grade_Level": "source.Grade_Level"
}).whenNotMatchedInsert(values={
    "Course_ID": "source.Course_ID",
    "Course_Name": "source.Course_Name",
    "Grade_Level": "source.Grade_Level"
}).execute()

# To get history and metrics (optional step for monitoring)
history_df = dim_deltacourse.history(1)  # Get the latest operation
operation_metrics = history_df.select("operationMetrics").collect()[0][0]

# Extract specific metrics
rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted)

# Printing results
print('Total rows of table: ', dim_deltacourse.toDF().count())
print("Merge Metrics:")
print(f"Rows inserted: {rows_inserted}")
print(f"Rows updated: {rows_updated}")
print(f"Rows deleted: {rows_deleted}")
print(f"Total rows affected: {rows_affected}")

StatementMeta(, ce999be1-aee6-4e03-a99c-854c62784cff, 3, Finished, Available, Finished)

NameError: name 'df_selected_dim_course' is not defined

In [20]:
%%sql

SELECT * FROM LH_gold.dim_course LIMIT 1000

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 21, Finished, Available, Finished)

<Spark SQL result set with 15 rows and 3 fields>

#### Loading data into Fact_student_performance

In [16]:
df_selected_Fact_student = (df.select( 
    "Student_ID",
    "Course_ID",
    "Enrollment_Date",
    "Completion_Date",
    "Status",
    "Final_Grade",
    "Attendance_Rate",
    "Time_Spent_on_Course_hrs",
    "Assignments_Completed",
    "Quizzes_Completed",
    "Forum_Posts",
    "Messages_Sent",
    "Quiz_Average_Score",
    "Assignment_Scores",
    "Assignment_Average_Score",
    "Project_Score",
    "Extra_Credit",
    "Overall_Performance",
    "Feedback_Score",
    "Completion_Time_Days",
    "Performance_Score",
    "Course_Completion_Rate",
    "Processing_Date"
))

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 17, Finished, Available, Finished)

In [17]:
from delta.tables import *

fact_student_table_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_gold.Lakehouse/Tables/fact_student_performance"
fact_deltastudent = DeltaTable.forPath(spark,fact_student_table_path)

# Perform the MERGE (UPSERT) operation and capture the operation metrics
merge_operation = fact_deltastudent.alias("target").merge(
    df_selected_Fact_student.alias("source"),
    "target.Student_ID = source.Student_ID AND target.Course_ID = source.Course_ID"
).whenMatchedUpdate(set={
    "Student_ID": "source.Student_ID",
    "Course_ID": "source.Course_ID",
    "Enrollment_Date": "source.Enrollment_Date",
    "Completion_Date": "source.Completion_Date",
    "Status": "source.Status",
    "Final_Grade": "source.Final_Grade",
    "Attendance_Rate": "source.Attendance_Rate",
    "Time_Spent_on_Course_hrs": "source.Time_Spent_on_Course_hrs",
    "Assignments_Completed": "source.Assignments_Completed",
    "Quizzes_Completed": "source.Quizzes_Completed",
    "Forum_Posts": "source.Forum_Posts",
    "Messages_Sent": "source.Messages_Sent",
    "Quiz_Average_Score": "source.Quiz_Average_Score",
    "Assignment_Scores": "source.Assignment_Scores",
    "Assignment_Average_Score": "source.Assignment_Average_Score",
    "Project_Score": "source.Project_Score",
    "Extra_Credit": "source.Extra_Credit",
    "Overall_Performance": "source.Overall_Performance",
    "Feedback_Score": "source.Feedback_Score",
    "Completion_Time_Days": "source.Completion_Time_Days",
    "Performance_Score": "source.Performance_Score",
    "Course_Completion_Rate": "source.Course_Completion_Rate",
    "Processing_Date": "source.Processing_Date"
}).whenNotMatchedInsert(values={
    "Student_ID": "source.Student_ID",
    "Course_ID": "source.Course_ID",
    "Enrollment_Date": "source.Enrollment_Date",
    "Completion_Date": "source.Completion_Date",
    "Status": "source.Status",
    "Final_Grade": "source.Final_Grade",
    "Attendance_Rate": "source.Attendance_Rate",
    "Time_Spent_on_Course_hrs": "source.Time_Spent_on_Course_hrs",
    "Assignments_Completed": "source.Assignments_Completed",
    "Quizzes_Completed": "source.Quizzes_Completed",
    "Forum_Posts": "source.Forum_Posts",
    "Messages_Sent": "source.Messages_Sent",
    "Quiz_Average_Score": "source.Quiz_Average_Score",
    "Assignment_Scores": "source.Assignment_Scores",
    "Assignment_Average_Score": "source.Assignment_Average_Score",
    "Project_Score": "source.Project_Score",
    "Extra_Credit": "source.Extra_Credit",
    "Overall_Performance": "source.Overall_Performance",
    "Feedback_Score": "source.Feedback_Score",
    "Completion_Time_Days": "source.Completion_Time_Days",
    "Performance_Score": "source.Performance_Score",
    "Course_Completion_Rate": "source.Course_Completion_Rate",
    "Processing_Date": "source.Processing_Date"
}).execute()


## History 

# Get the history of the Delta table to extract metrics
history_df = fact_deltastudent.history(1)  # Get the latest operation

# Extract metrics from the history DataFrame
operation_metrics = history_df.select("operationMetrics").collect()[0][0]

# Extract specific metrics
rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
rows_copied = operation_metrics.get('numTargetRowsCopied', 0)
rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted) + int(rows_copied)

print('Total rows of table: ',fact_deltastudent.toDF().count())
print("Merge Metrics:")
print(f"Rows inserted: {rows_inserted}")
print(f"Rows updated: {rows_updated}")
print(f"Rows deleted: {rows_deleted}")
#print(f"Rows copied: {rows_copied}")
print(f"Total rows affected: {rows_affected}")

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 18, Finished, Available, Finished)

Total rows of table:  15
Merge Metrics:
Rows inserted: 15
Rows updated: 0
Rows deleted: 0
Total rows affected: 15


In [18]:
%%sql

SELECT * FROM LH_gold.fact_student_performance LIMIT 1000

StatementMeta(, 9e247b07-bcb9-4f91-8c9b-219d0f66a938, 19, Finished, Available, Finished)

<Spark SQL result set with 15 rows and 23 fields>

In [7]:
df_course= spark.read.format('delta').load(f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_gold.Lakehouse/Tables/dim_course")

StatementMeta(, 00754576-9c74-4bba-b382-60deecc09c92, 9, Finished, Available, Finished)

In [8]:
df_nodups = df_course.dropDuplicates()

StatementMeta(, 00754576-9c74-4bba-b382-60deecc09c92, 10, Finished, Available, Finished)

In [9]:
display(df_nodups)

StatementMeta(, 00754576-9c74-4bba-b382-60deecc09c92, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fd1b1a6f-37bb-4458-81d5-69ed759e0ef7)

In [10]:
df_nodups.write.format('delta').mode('overwrite').saveAsTable('dim_course')

StatementMeta(, 00754576-9c74-4bba-b382-60deecc09c92, 12, Finished, Available, Finished)

In [11]:
df = spark.sql("SELECT * FROM LH_Gold.dim_course LIMIT 1000")
display(df)

StatementMeta(, 00754576-9c74-4bba-b382-60deecc09c92, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fc209d0d-cdff-4b87-898b-b6fbf106db24)